# **Импорт библиотек**

In [28]:
import pandas as pd
import json
import dill
import os
import warnings

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# **Загрузка и подготовка данных**

In [29]:
# --- Путь к файлу данных ---
data_path = 'Сводная_2.xlsx'

In [30]:
try:
    if data_path.endswith('.xlsx'):
        data = pd.read_excel(data_path)

    elif data_path.endswith('.csv'):
        data = pd.read_csv(data_path)

except Exception as e:
    raise ValueError(f"Ошибка при загрузке данных: {e}")

In [31]:
def preprocess_df(df: pd.DataFrame, threshold_percent: float = 99, del_first_row: bool = True) -> pd.DataFrame:
    """
    Очищает DataFrame, удаляя столбцы с пропусками выше заданного порога и (опционально) первую строку.

    Функция выполняет следующие действия:
    1. Удаляет первую строку, если параметр `del_first_row` равен `True`.
    2. Удаляет столбцы, в которых процент пропущенных значений (`NaN`) превышает заданный порог.
    3. Заполняет оставшиеся пропуски пустыми строками.

    Args:
        df (pd.DataFrame): Входной DataFrame для обработки.
        threshold_percent (float, optional): Порог для удаления столбцов (в процентах).
            Столбцы с процентом пропусков выше этого значения будут удалены.
            Должен быть в диапазоне от 0 до 100. По умолчанию 99.
        del_first_row (bool, optional): Флаг, указывающий, нужно ли удалять первую строку.
            По умолчанию `True`.

    Returns:
        pd.DataFrame: Очищенный DataFrame, в котором:
            - Удалены столбцы с пропусками выше порога.
            - Удалена первая строка (если `del_first_row=True`).
            - Оставшиеся пропуски заполнены пустыми строками.
    """
    if not (0 <= threshold_percent <= 100):
        raise ValueError("Порог должен быть в диапазоне от 0 до 100.")

    df = df.iloc[1:].reset_index(drop=True) if del_first_row else df.copy()

    threshold = threshold_percent / 100
    columns_to_keep = df.columns[df.isnull().mean() <= threshold]

    return df[columns_to_keep].fillna("")


def replace_rare_values(data: pd.DataFrame, column_name: str = 'Группа МТР (ред.)', 
                        threshold: int = 100, replacement_value="Необходимо дообучение"):
    """
    Заменяет редкие значения в указанной колонке датафрейма на заданное значение.

    Редкими считаются значения, количество которых в колонке меньше заданного порога (threshold).
    Функция возвращает копию датафрейма с замененными значениями.

    Args:
        data (pd.DataFrame): Исходный датафрейм, в котором будут заменены редкие значения.
        column_name (str, optional): Название колонки для замены редких значений.
            По умолчанию 'Группа МТР (ред.)'.
        threshold (int, optional): Пороговое значение для определения редких значений.
            Если количество уникальных значений в колонке меньше этого числа, они будут заменены.
            По умолчанию 100.
        replacement_value (str, optional): Значение, на которое будут заменены редкие значения.
            По умолчанию "Необходимо дообучение".

    Returns:
        pd.DataFrame: Копия исходного датафрейма с замененными редкими значениями в указанной колонке.
    """
    data_copy = data.copy()
    values_count = data_copy[column_name].value_counts()
    values_to_replace = list(values_count[values_count < threshold].index)
    data_copy[column_name] = data_copy[column_name].apply(
        lambda x: replacement_value if x in values_to_replace else x
    )

    return data_copy


data = preprocess_df(data)
data = replace_rare_values(data)

# **Обучение модели**

### Создание json конфига для модели

In [32]:
# ---------------------------
# Ручное определение колонок (лучшие колонки после тестировавния)
# ---------------------------

TARGET_COL = "Группа МТР (ред.)"
categorical_cols = ['Чертёж', 'Тип, марка ГОСТ МТР', 'Марка РД']
text_cols = ['Наименование МТР', 'Ед. изм.']

# ---------------------------------------------------------------------------------------
# Автоматическое определение категориальных колонок на основе кол-ва уникальных значений
# ---------------------------------------------------------------------------------------

# TARGET_COL = "Группа МТР (ред.)"
# CATEGORICAL_THRESHOLD = 150  # Порог уникальных значений для категориальных данных

# # Разделение колонок на категориальные и текстовые
# categorical_cols = [
#     col for col in data.columns 
#     if col != TARGET_COL and data[col].nunique() <= CATEGORICAL_THRESHOLD
# ]

# text_cols = [
#     col for col in data.columns 
#     if col != TARGET_COL and data[col].nunique() > CATEGORICAL_THRESHOLD
# ]

# print("Категориальные столбцы:", categorical_cols)
# print("Текстовые столбцы:", text_cols)

In [35]:
# ----------------------------------------------------------------------
# - Словарь сопоставления предпологаемых колонок с колонками в данных. -
# ----------------------------------------------------------------------

# Создаём маппинг по текущим данным
mapping = {i:i for i in data.columns if i != TARGET_COL}

# Ручной ввод:
# mapping = {
#         "Чертёж": ["Чертёж часть 1", "Чертёж часть 2"], # Например, столбец "Чертёж" разделён на 2 столбца
#         "Марка РД": "Марка РД",
#         "Наименование МТР": "Наименование МТР",
#         "Тип, марка ГОСТ МТР": "Тип, марка ГОСТ МТР",
#         "Ед. изм.": "Ед. изм."
#     }

In [36]:
# Финальный конфиг
config = {
    "column_mapping": mapping,
    "categorical_cols": categorical_cols,
    "text_cols": text_cols,
}

# Сохранение в JSON файл
config_path = "models/model_mtp_group_config.json"
with open(config_path, "w", encoding="utf-8") as f:
    json.dump(config, f, ensure_ascii=False, indent=4)

### Обучение модели

In [37]:
class SafeColumnMapper(BaseEstimator, TransformerMixin):
    """
    Комбинированный класс для безопасного переиндексирования и маппинга колонок в DataFrame.

    Атрибуты:
        required_cols (list): Список колонок, которые должны присутствовать в выходном DataFrame.
        fill_value (any, optional): Значение, которым будут заполнены отсутствующие колонки (по умолчанию None).
        mapping (dict): Словарь, где ключ – ожидаемое имя столбца, а значение – либо имя столбца во входных данных, либо список имен, которые нужно объединить.
    """
    def __init__(self, required_cols=None, fill_value=None, mapping=None):
        self.required_cols = required_cols if required_cols is not None else []
        self.fill_value = fill_value
        self.mapping = mapping if mapping is not None else {}

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Преобразуем в DataFrame, если входные данные не DataFrame
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)

        # Применяем маппинг колонок
        df_new = pd.DataFrame(index=X.index)
        for target_col, source_cols in self.mapping.items():
            if isinstance(source_cols, list):
                valid_cols = [col for col in source_cols if col in X.columns]
                missing = [col for col in source_cols if col not in X.columns]

                if missing:
                    warnings.warn(f"Следующие столбцы отсутствуют в переданных данных и будут пропущены: {missing}", UserWarning)

                df_new[target_col] = X[valid_cols].fillna("").astype(str).agg(" ".join, axis=1) if valid_cols else ""

            elif isinstance(source_cols, str):
                if source_cols in X.columns:
                    df_new[target_col] = X[source_cols] if source_cols in X.columns else ""

            else:
                raise ValueError(f"Некорректный тип значения в mapping для {target_col}: {source_cols} ({type(source_cols)})")

        # Добавляем отсутствующие колонки
        missing_cols = [col for col in self.required_cols if col not in df_new.columns]
        if missing_cols:
            warnings.warn(f"Необходимые столбцы не найдены и будут заполнены значением {repr(self.fill_value)}: {missing_cols}", UserWarning)

        df_new = df_new.reindex(columns=self.required_cols, fill_value=self.fill_value).fillna("")

        return df_new


def create_pipeline(mapping, categorical_cols, text_cols, fill_value=""):
    """
    Создаёт пайплайн, который сначала приводит входные данные к ожидаемому формату
    с помощью SafeColumnMapper, гарантируя наличие всех требуемых столбцов и
    выполняя предварительную обработку, а затем проводит классификацию.

    Аргументы:
        mapping: dict, сопоставление ожидаемых названий столбцов с именами из входных данных.
        categorical_cols: список ожидаемых имен столбцов для категориальных признаков.
        text_cols: список ожидаемых имен столбцов для текстовых признаков.
        fill_value: значение для заполнения отсутствующих колонок.
    """
    # Шаг приведения входных данных к нужному виду
    safe_column_mapper = SafeColumnMapper(required_cols=categorical_cols + text_cols, fill_value=fill_value, mapping=mapping)

    # Пайплайн для категориальных признаков
    cat_pipeline = Pipeline([
        ('astype_str', FunctionTransformer(lambda x: x.astype(str), validate=False)),
        ("onehot", OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
    ])

    # Пайплайн для текстовых признаков
    text_pipeline = Pipeline([
        # Преобразуем входной массив в DataFrame с колонками text_cols
        ("to_df", FunctionTransformer(lambda arr: pd.DataFrame(arr, columns=text_cols), validate=False)),
        # Объединяем колонки в единый текст
        ("combine_text", FunctionTransformer(lambda df: df.apply(lambda row: " ".join(row.astype(str)), axis=1), validate=False)),
        ("tfidf", TfidfVectorizer())
    ])

    # ColumnTransformer, который параллельно обрабатывает категориальные и текстовые признаки
    preprocessor = ColumnTransformer([
        ("cat", cat_pipeline, categorical_cols),
        ("text", text_pipeline, text_cols)
    ])

    # Итоговый пайплайн
    pipeline = Pipeline([
        ("safe_column_mapper", safe_column_mapper),
        ("preprocessing", preprocessor),
        ("clf", RandomForestClassifier(random_state=42))
    ])

    return pipeline

In [38]:
# Загрузка конфигурации
with open(config_path, "r", encoding="utf-8") as f:
    loaded_config = json.load(f)

pipeline = create_pipeline(mapping, categorical_cols, text_cols, fill_value="")

# Разделяем на признаки и целевую переменную
X = data.drop(columns=[TARGET_COL])
y = data[TARGET_COL]

# Тренировочная и тестовая выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

# Обучаем
pipeline.fit(X_train, y_train)

# Предсказываем
y_pred = pipeline.predict(X_test)

# Оцениваем результаты
print(classification_report(y_test, y_pred))

                                                         precision    recall  f1-score   support

                                  Арматура строительная       0.99      0.99      0.99       183
                                                  Бетон       1.00      0.99      0.99        70
                                            Воздуховоды       1.00      1.00      1.00        63
                                                    ЖБИ       1.00      0.98      0.99        92
                                                    ЗРА       0.98      0.97      0.98       183
                                     Инертные материалы       0.90      1.00      0.95        46
                                  КИПиА и комплектующие       0.99      0.89      0.94       167
                                  Кабеленесущие системы       0.81      0.91      0.86       151
                                Кабельная арматура (КА)       0.95      0.95      0.95       210
                       Кабель

### Сохранение модели

In [39]:
# Сохранение модели
save_path = "models/model_mtp_group.pkl"

# Создание папки, если её нет
os.makedirs(os.path.dirname(save_path), exist_ok=True)

with open(save_path, "wb") as f:
    dill.dump(pipeline, f)